In [17]:
# Important modules used
import ipywidgets as widgets
from IPython.display import display
from scipy.special import erf
from scipy.sparse import diags
from scipy.integrate import odeint
import matplotlib.pyplot as plt
import numpy as np

# finite fourier transform function
def fft(x, t, n):
    f = 1 - x  # Steady state solution
    psi = 0  # Set inital summation to zero
    for i in range(1, n+1):  # sums the solution to n terms
        psi += (np.sin(i*np.pi*x)/(i*np.pi))*np.exp(-(i*np.pi)**2*t)
    return f - 2*psi

# Similarity solution funciton
def simSln(x, t):
    eta = x/(2*np.sqrt(t))  # Calculates Similarity variable
    return 1 - erf(eta)

# Method of lines solver (solves PDEs)
def MOL(x,t):
    n = x.size  # finds domain size for matrix
    delta_x = x[1]-x[0]
    # generates sparse matrix   
    diagonals = [-2, 1, 1]
    offsets = [0, -1, 1]
    A = diags(diagonals, offsets, shape=(n, n), format='csr')
    # converts sparse matrix into a full array
    A = A.toarray()
    # Setting up matrix to be solve as an ODE
    A[0,:] = 0
    A[n-1,:] = 0
    # scales matrix by 1/delta x^2 because of finite difference    
    A = (1/delta_x**2)*A
    inital = np.zeros(x.size)
    time = np.linspace(0,t,x.size)
    b = 1  # this b is only add so function so (A,b) is a tuple 
    # Derivative of theta with respect to time as defined by finite difference
    def theta_dt(theta,t,A,b):
        # Sets up boundary conditions
        theta[0] = 1
        theta[-1] = 0
        return np.dot(A,theta)

    # Solves the system of equation using scipy ODEint
    theta = odeint(theta_dt, inital, time, args=(A,b))
    theta_final = theta[-1]  # Uses only the last time point for graphing
    return theta_final
    
# Makes slider longer
slider_layout = widgets.Layout(width='500px')
# Create sliders
n_slider = widgets.IntSlider(value=2, min=1, 
    max=10, step=1, description='n:',layout=slider_layout)
t_slider = widgets.FloatSlider(value=0.010, min=0.0001,
    max=0.5, step=0.0001, description='t:',layout=slider_layout)

# Create output widget for the plot
output = widgets.Output()

# Function to handle slider value changes and update the plot
def update_plot(change):
    # This is what updates the plot every time the slider is moved
    with output:
        output.clear_output(wait=True)
        n = n_slider.value
        t = t_slider.value
        # Defines the domain and runs the 3 solutions
        x = np.linspace(0, 1, 200)
        y1 = fft(x, t, n)
        y2 = simSln(x, t)
        y3 = MOL(x, t)
        # Plotting stuff
        plt.figure(figsize=(10, 6), dpi=300)
        #plt.plot(x, y1, color='DarkRed', label='FFT')
        plt.plot(x, y2, color='Coral', label='Similarity Solution')
        plt.plot(x, y3, color='Indigo', linestyle = ':', label='Numerical Solution')
        plt.xlabel('x', fontsize=12)
        plt.ylabel('θ', fontsize=14, 
            rotation=0, labelpad=15)
        plt.ylim(-0.1, 1.2)
        plt.legend()
        plt.tight_layout()
        plt.show()

# Observe slider value changes
n_slider.observe(update_plot, names='value')
t_slider.observe(update_plot, names='value')

# Create a vertical box layout
sliders = widgets.VBox([n_slider, t_slider])

# Display sliders and output
display(sliders, output)

# Initial plot
update_plot(None)

Output()